# Zonal Exchange Network
ZEN（Zonal Exchange Network） 是一个原创的水文模型，用于模拟流域内的水文过程。ZEN使用神经网络来描述产汇流过程。

In [1]:
# 自动加载模块
%load_ext autoreload
%autoreload 2
# 清除所有变量
from hydronetwork.utils import clear_all

clear_all()

ImportError: cannot import name 'clear_all' from 'hydronetwork.utils' (C:\Users\18313\PycharmProjects\HydroNetwork\hydronetwork\utils.py)

## 1. 数据处理

In [20]:
from hydronetwork.data.camels_us import load_timeseries
from hydronetwork.dataset.preprocessing import split_timeseries
from hydronetwork.dataset.dataset import get_dataset

lookback = 365
horizon = 7
timeseries = load_timeseries(gauge_id='13023000', unit="mm/day")
timeseries.columns

Index(['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)',
       'streamflow'],
      dtype='object', name='timeseries_type')

对['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)']进行归一化

In [21]:
from hydronetwork.dataset.preprocessing import normalize

timeseries_need_normalize = timeseries[['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)']]
timeseries_normalized = normalize(timeseries_need_normalize)
timeseries = timeseries.drop(columns=['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'])
timeseries = timeseries.join(timeseries_normalized)

构建数据集

In [22]:
# train_data, test_data = split_timeseries(timeseries, split_list=[0.8, 0.2])
# train_dataset = get_dataset(train_data, 
#                             lookback=lookback, 
#                             horizon=horizon, 
#                             batch_size=2048, 
#                             features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
#                             target="streamflow"
#                             )
# test_dataset = get_dataset(test_data,
#                             lookback=lookback,
#                             horizon=horizon,
#                             batch_size=2048,
#                             features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
#                             target="streamflow"
#                             )
dataset = get_dataset(timeseries,
                      lookback=lookback,
                      horizon=horizon,
                      batch_size=1024,
                      features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
                      target="streamflow"
                      )

## 2. 模型构建

In [23]:
from hydronetwork.model import ZonalExchangeNet

model = ZonalExchangeNet(m=3,
                         n=128,
                         n_mix_steps=3,
                         water_capacity_max=120,
                         layer_units=[32, 16, 1],
                         horizon=horizon, )

## 3. 模型训练

In [24]:
from hydronetwork.train import WarmupExponentialDecay, callback_for_features_selection
from hydronetwork.evaluate.metrics import rmse, nse
from keras.src.optimizers import AdamW

model.compile(optimizer=AdamW(learning_rate=WarmupExponentialDecay(dataset_length=len(dataset),
                                                                   initial_learning_rate=1e-4, )),
              loss=nse,
              metrics=[rmse]
              )
model.fit(dataset,
          epochs=500,
          verbose=1,
          shuffle=True,
          callbacks=callback_for_features_selection())

Epoch 1/500
 5/13 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - loss: 96.5390 - rmse: 8.2017 

KeyboardInterrupt: 